In [1]:
# перезаписать файл .json в .csv
import pandas as pd

df = pd.read_json('train.json')
df.to_csv('train.csv', index=False)

df = pd.read_json('test.json')
df.to_csv('test.csv', index=False)

In [2]:
# добавить столбец 'cuisine' в test 
import pandas as pd

# Загрузка CSV
df = pd.read_csv('test.csv')

# Добавление пустого столбца на позицию `index=1` (второй столбец)
df.insert(1, 'cuisine', None)  # или pd.NA, "", np.nan

# Сохранение
df.to_csv('test.csv', index=False)

In [3]:
# объединяем два csv в один
import pandas as pd

# Загружаем оба CSV
df1 = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')

# Объединяем их по строкам
df_combined = pd.concat([df1, df2], ignore_index=True)

# Сохраняем результат в новый CSV
df_combined.to_csv('combined-max.csv', index=False)

# Выводим первые 5 строк для проверки
print(df_combined.head())


      id      cuisine                                        ingredients
0  10259        greek  ['romaine lettuce', 'black olives', 'grape tom...
1  25693  southern_us  ['plain flour', 'ground pepper', 'salt', 'toma...
2  20130     filipino  ['eggs', 'pepper', 'salt', 'mayonaise', 'cooki...
3  22213       indian        ['water', 'vegetable oil', 'wheat', 'salt']
4  13162       indian  ['black pepper', 'shallots', 'cornflour', 'cay...


In [4]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
# Загружаем данные из CSV
df = pd.read_csv('combined.csv')

# Проверим названия столбцов
print(df.columns)

# Преобразуем строки в списки ингредиентов
df['ingredients'] = df['ingredients'].apply(lambda x: x.split(', ') if isinstance(x, str) else [])

# Применяем MultiLabelBinarizer для one-hot encoding
mlb = MultiLabelBinarizer()
ingredients_onehot = mlb.fit_transform(df['ingredients'])

# Преобразуем в DataFrame
ingredients_df = pd.DataFrame(ingredients_onehot, columns=mlb.classes_)

# Убираем лишние пробелы в столбцах
df.columns = df.columns.str.strip()

# Объединяем one-hot признаки с оригинальными данными, удаляя столбец ingredients
df_onehot = pd.concat([ingredients_df, df.drop(['ingredients'], axis=1)], axis=1)

# Признаки (X) — это все столбцы, кроме 'cuisine'
X = df_onehot.drop('cuisine', axis=1)

# Целевая переменная (y) — это столбец 'cuisine'
y = df_onehot['cuisine']

# Преобразуем целевую переменную (cuisine) в числовой формат
y = pd.factorize(y)[0]  # Преобразует категориальные значения в числа (например, 'french' -> 0)

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


Index(['id', 'cuisine', 'ingredients'], dtype='object')


In [5]:
# 2. Классификатор CatBoost.
!pip install catboost

You should consider upgrading via the '/home/dzmitry/.pyenv/versions/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


In [6]:
# Классификатор CatBoostClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix



# Создание и обучение классификатора CatBoost
clf = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=3, random_state=42, verbose=0)
clf.fit(X_train, y_train)

# Прогнозируем на тестовых данных
y_pred = clf.predict(X_test)  # <- Исправлено: clf вместо model

# 1. Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# 2. Матрица ошибок (Confusion Matrix)
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

print(classification_report(y_test, y_pred))

Accuracy: 0.36

Confusion Matrix:
[[33  0  1  0  5  0  0  7  2  2  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [15  0  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [12  0  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [16  0  0  0  0  0  0 26  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [20  0  0  0  0  0  0  0 16  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  0  0  0  0  0  0  0  0  8  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 6  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  0  0  0  0  0  0  

/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

In [7]:
# Классификатор ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Создание и обучение классификатора Extra Trees
clf = ExtraTreesClassifier(n_estimators=100, max_features='sqrt', random_state=42)
clf.fit(X_train, y_train)

# Прогнозирование и оценка точности
y_pred = clf.predict(X_test)

# 1. Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# 2. Матрица ошибок (Confusion Matrix)
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

print(classification_report(y_test, y_pred))


Accuracy: 0.40

Confusion Matrix:
[[17  0  2  0  7  0  0 15  5  3  0  0  0  0  1  0  0  0  0  0  0]
 [ 2  1  0  0  0  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [10  0  5  0  0  0  0  1  2  0  0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1  0  1  1  0  0  0  0  0  0  0  0]
 [ 6  0  0  0  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  3  0  0  0  0  0  0  0  1  0  0  0  0  0]
 [ 7  0  1  0  0  0  0 34  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [10  0  1  0  1  0  0  1 23  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  0  0  8  0  0  0  0  0  0  1  0  2  0  0]
 [ 1  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  0  0  3  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  1  0  1  0  0  0  0  0  0  1  0  0]
 [ 3  0  0  0  0  0  0  3  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  1  0  0  0  0  0  

/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

In [8]:
# классификатор AdaBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


# Определение базовой модели (слабой)
base_model = DecisionTreeClassifier(max_depth=1)  # Дерево решений с 1 разделением

# Инициализация AdaBoost
adaboost = AdaBoostClassifier(
    estimator=base_model,
    n_estimators=50,  # Количество слабых моделей
    learning_rate=1.0,  # Скорость обучения
    random_state=42
)

# Обучение модели
adaboost.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = adaboost.predict(X_test)

# 1. Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# 2. Матрица ошибок (Confusion Matrix)
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

print(classification_report(y_test, y_pred))


Accuracy: 0.24

Confusion Matrix:
[[43  0  0  0  4  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [17  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 8  0  0  0  3  0  0  4  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [37  0  0  0  0  0  0  5  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [31  0  0  0  0  0  0  0  5  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 8  0  0  0  0  0  0  0  0  5  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  0  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  0  0  0  0  0  0  

/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

In [9]:
# классификатор KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Создание модели (k=3 по умолчанию)
knn = KNeighborsClassifier(n_neighbors=3)

# Обучение модели
knn.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = knn.predict(X_test)

# 1. Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# 2. Матрица ошибок (Confusion Matrix)
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

print(classification_report(y_test, y_pred))

Accuracy: 0.12

Confusion Matrix:
[[22  2  7  1  6  1  0 10  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  1  0  0  0  0  2  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [11  0  2  0  1  0  0  4  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 5  0  4  1  0  0  0  3  1  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 1  1  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 4  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [23  3  3  0  2  1  0  5  3  1  0  0  0  0  0  0  1  0  0  0  0]
 [19  1  5  0  2  1  0  6  2  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 3  0  1  0  0  0  0  4  5  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  1  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  1  0  1  0  0  1  0  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  1  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 2  0  1  0  1  0  0  2  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  

/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

In [10]:
# классификатор QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Обучение QDA
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)

# Прогнозируем на тестовых данных
y_pred = qda.predict(X_test)  

# 1. Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# 2. Матрица ошибок (Confusion Matrix)
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

print(classification_report(y_test, y_pred))

/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 0 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 1 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 2 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 3 is not full rank. Increasing th

Accuracy: 0.08

Confusion Matrix:
[[ 1  0  5  0  2  0  0  4  0  2  0  0  0  0  0  2 33  0  0  1  0]
 [ 0  0  1  0  0  0  0  2  0  0  0  0  0  0  0  0  3  0  0  0  0]
 [ 0  0  4  0  2  0  0  1  0  1  1  0  0  1  0  0  9  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  3  0  0  0  0]
 [ 0  0  0  0  2  0  0  0  0  2  0  0  0  1  0  0 10  0  0  0  0]
 [ 0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  1  0  1  1  0  0  0  0  0  0  0  1  1  0  0  0  0]
 [ 0  0 11  0  2  0  0  4  0  3  0  0  0  0  0  0 21  0  0  1  0]
 [ 0  0  3  0  3  0  0  3  0  3  0  0  0  1  0  1 21  0  0  1  0]
 [ 0  0  1  0  2  0  0  3  0  0  0  0  0  0  0  0  7  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  3  0  0  0  0]
 [ 0  0  1  0  0  0  0  3  0  0  0  0  0  0  0  0  2  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  2  1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  6  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  

/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dzmitry/.pyenv/versions/3.10.0/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap